In [19]:
import math
import re
import heapq
import csv

import pandas as pd
import numpy as np

import time

In [43]:
import numpy.linalg as LA

from IPython.display import clear_output
from fingerprint.fingerprint import get_fingerprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


## Settings

In [45]:
input_filename = 'data/unique_sorted_split.txt' 
output_filename = 'output/clustered.txt'
mapping_filename = 'output/mapping_clusters.csv'
threshold = 0.85


## Load data

In [40]:
df = pd.read_csv(input_filename, names=["Organization"], quoting=csv.QUOTE_NONE)


## Run Tf Idf Clustering

In [50]:
outfile = open(output_filename, "w")
mapping = open(mapping_filename, "w")

vectorizer = TfidfVectorizer()
transformer = TfidfTransformer()
corpus = set()

start = time.time()

for i, s in df.iterrows():
    if i % 10000 == 0:
        clear_output()
        print(f"{i} / {df.shape[0]}\nElapsed: {time.time()-start}s")
        
    original_string = s.Organization
    cleaned_string = get_fingerprint(original_string)
    
    if len(cleaned_string).split() < 3:
        outfile.write("\n")
        mapping.write(f"{original_string},\n")
        continue
    elif not corpus: # For first iteration
        label = cleaned_string
        corpus = set()
        corpus.update(cleaned_string.split())
    else:
        train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
        test = vectorizer.transform([cleaned_string]).toarray() # Should this be original or cleand label?
        score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]

        if np.isnan(score) or score < threshold: # Organization is not part of previous cluster
            corpus = set()
            corpus.update(cleaned_string.split())
            train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
            test = vectorizer.transform([cleaned_string]).toarray() # Should this be original or cleand label?
            score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]
            label = cleaned_string

    outfile.write(label + "\n")
    mapping.write(f"{original_string},{label}\n")

outfile.close()
mapping.close()

80000 / 3261955
Elapsed: 309.17405796051025


KeyboardInterrupt: 